In [2]:
from collections import Counter
import pickle
import os
import yaml
import random
import numpy as np
import pandas as pd

from scipy.stats import bernoulli
from sklearn import decomposition

import bkcharts
import bokeh.io
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.layouts import row
bokeh.io.output_notebook()

os.chdir(os.path.expanduser("~/projects/embedding/src"))
from probe2vec.w2v import word2vec, Word2VecEmbedder
from probe2vec.dataset_reader import kmerize_fastq_parse, kmerize_fasta_parse, DatasetReader
from probe2vec.embedding_utils import build_index, most_similar, merge_counters, reshape_to_vector
from probe2vec.theano_minibatcher import (
    TheanoMinibatcher, NoiseContrastiveTheanoMinibatcher
)

Loading BokehJS ...

In [6]:
# get the model file params
dim = '50'
config_dir = os.path.expanduser('~/projects/embedding/src/config_yamls/embedding_dim_exps/')
results_dir = os.path.expanduser('~/projects/embedding/results/embedding_size/k_8_emb_' + dim)
model_yaml = 'k_8_emb_' + dim + '.yaml'
with open(os.path.join(config_dir, model_yaml)) as f:
    params = yaml.load(f)

In [7]:
# load the embedder
data_dir = os.path.abspath(params['data_dir'])
selex_files = [os.path.join(data_dir,f) for f in os.listdir(data_dir) if f.endswith(params['file_suffixes'])]
os.chdir(os.path.expanduser("~/projects/embedding/src"))

In [8]:
if "fastq" in params['parser']:
    parser = kmerize_fastq_parse
else:
    parser = kmerize_fasta_parse
    

# load the DatasetReader object from the save dir
reader = DatasetReader(files=[], directories=[], skip=[], noise_ratio=15, 
                      t=1e-5, num_processes=3, 
                      unigram_dictionary=None, 
                      min_frequency=0, kernel=[1, 2, 3, 
                      4, 5, 5, 4, 3, 2, 1], 
                      load_dictionary_dir=params['save_dir'], 
                      max_queue_size=0, 
                      macrobatch_size=20000, 
                      parse=parser, 
                      verbose=True, k=params['K'], 
                      stride=params['stride'])
    
# load the embedder, DatasetReader objects
batch_size = 1000
noise_ratio=15
num_embedding_dimensions=params['num_embedding_dimensions']
full_batch_size = batch_size * (1 + noise_ratio)

minibatcher = NoiseContrastiveTheanoMinibatcher(
    batch_size=batch_size,
    noise_ratio=noise_ratio,
    dtype="int32",
    num_dims=2
)

embedder = Word2VecEmbedder(input_var=minibatcher.get_batch(),
                            batch_size=full_batch_size,
                            vocabulary_size=reader.get_vocab_size(),
                            num_embedding_dimensions=num_embedding_dimensions)
embedder.load(os.path.join(params['save_dir'],''))
params['save_dir']

Loading dictionary from ../results/embedding_size/k_8_emb_50...
pruning dictionary to eliminate tokens occuring less than 0 times.
dropped  0  tokens in pruning the unigram dictionary


'../results/embedding_size/k_8_emb_50'

In [9]:
# Load a random set of probes from a randomly chosen factor, embed, and visualize the probes
def sample_from_factor(filename, data_dir, percentage=0.1):
    factor = filename.split('/')[-1]
    factor = factor.split('_')[0]

    tokenized_sentences = kmerize_fasta_parse(filename, **params) 
    samples = bernoulli.rvs(percentage,size=len(tokenized_sentences)).astype('bool').tolist()

    return factor, [s for s,i in zip(tokenized_sentences,samples) if i]
    

In [10]:
random.shuffle(selex_files)
factor, sample_sentences = sample_from_factor(selex_files[0], data_dir)
other_sentences = []
other_factors = []
other_factors_by_kmer = []
files_indices = np.random.random_integers(1, len(selex_files) - 1,10).tolist()
for i in files_indices:
    f, s = sample_from_factor(selex_files[i], data_dir, 0.01)
    other_sentences.extend(s)
    other_factors.extend([f] * len(s))
    other_factors_by_kmer.extend([f] * (len(s) * len(s[0])))

/Users/zamparol/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: This function is deprecated. Please call randint(1, 175 + 1) instead
  


In [11]:
# Embed each of the k-mers from the sample sentences, and the other sentences
embedded_samples = []
embedded_samples_probe_mean = []
for sentence in sample_sentences:
    sentence_token_ids = [reader.unigram_dictionary.get_id(token) for token in sentence]
    embedded_tokens = [embedder.embed(t) for t in sentence_token_ids]
    embedded_samples.append(embedded_tokens)
    embedded_samples_probe_mean.append(np.concatenate(embedded_tokens).mean(axis=0))
    
embedded_others = []
embedded_others_probe_mean = []
for sentence in other_sentences:
    sentence_token_ids = [reader.unigram_dictionary.get_id(token) for token in sentence]
    embedded_tokens = [embedder.embed(t) for t in sentence_token_ids]
    embedded_others.append(embedded_tokens)
    embedded_others_probe_mean.append(np.concatenate(embedded_tokens).mean(axis=0))
    
embedded_samples_flat = [val for sublist in embedded_samples for val in sublist]
embedded_others_flat = [val for sublist in embedded_others for val in sublist]

In [12]:
# Pack into numpy arrays
embedded_samples_array = np.concatenate(embedded_samples_flat)
embedded_others_array = np.concatenate(embedded_others_flat)

embedded_samples_probes = np.stack(embedded_samples_probe_mean)
embedded_others_probes = np.stack(embedded_others_probe_mean)

In [13]:
# apply PCA transformation to both sets
all_pts = np.concatenate([embedded_samples_array,embedded_others_array])
pca = decomposition.PCA(n_components=2)
pca.fit(all_pts)
all_pts_2pcs = pca.transform(all_pts)

all_probes = np.concatenate([embedded_samples_probes, embedded_others_probes])
pca.fit(all_probes)
all_probes_2pcs = pca.transform(all_probes)

In [14]:
# k-mer embedded data into DataFrame
kmer_labels = [factor] * len(embedded_samples_flat)
kmer_other_labels = other_factors_by_kmer
kmer_labels.extend(kmer_other_labels)

kmer_d = {'PC1': all_pts_2pcs[:,0], 'PC2': all_pts_2pcs[:,1], 'Factor': kmer_labels}
my_kmer_df = pd.DataFrame(data=kmer_d)
embedded_kmers_2d = ColumnDataSource(my_kmer_df)

# probe embedded data into DataFrame
probe_labels = [factor] * len(sample_sentences)
probe_labels.extend(other_factors)
d = {'PC1': all_probes_2pcs[:,0], 'PC2': all_probes_2pcs[:,1], 'Factor': probe_labels}
my_probe_df = pd.DataFrame(data=d)
embedded_probes_2d = ColumnDataSource(my_probe_df)

print("Factor is :", factor)

from bkcharts import Scatter, show
kmers_2pcs = Scatter(my_kmer_df, x='PC1', y='PC2', color='Factor', marker='Factor',
            title="PCA plot for embedded k-mers", legend="top_right",
            xlabel="PC1", ylabel="PC2")
kmers_2pcs.title.align = "center"

probes_2pcs = Scatter(my_probe_df, x='PC1', y='PC2', color='Factor', marker='Factor',
            title="PCA plot for embedded probes", legend="top_right",
            xlabel="PC1", ylabel="PC2")
probes_2pcs.title.align = "center"

show(row(kmers_2pcs, probes_2pcs))

Factor is : HOXB13
